In [1]:
import Pkg
#Pkg.add("JuMP")
#Pkg.add("GLPK")
#Pkg.add("Gruobi")
#Pkg.add("DataFrames")
#Pkg.add("CSV")
# Pkg.add("PyPlot")
# Pkg.add("PyCall")

In [2]:
using JuMP, GLPK
using DataFrames
using CSV
using PrettyTables
using Random
using Plots
# using PyPlot
# using PyCall

In [3]:
# Load the data
scenarios_df = CSV.read("../data/scenarios.csv", DataFrame)

n_scenarios = size(scenarios_df, 2)/3
n_scenarios = convert(Int, n_scenarios)

# create a dictonary with 200 dataframes for each scenario
all_scenarios = Dict()
for i in 1:n_scenarios
    df_helper = DataFrame(scenarios_df[:,3*i-2:3*i])
    df_helper[!,3] = df_helper[!,3] .* 1.0
    rename!(df_helper, [:"price", :"wind power", :"grid_excess"])
    all_scenarios[i] = df_helper
end

In [4]:
W = 250
hours = 24
Random.seed!(123)
selected_scenarios = rand(1:n_scenarios, W)

scenarios = Dict()
counter = 1
for i in selected_scenarios
    scenarios[counter] = all_scenarios[i]
    counter += 1
end

alpha = 0.9
#make beta a list of values
betas = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
objectiv_values = []
exp_profits = []
CVaR_values = [] 

Any[]

In [5]:
# create a dataframe to store the profits per scenario for each beta
profits_df = DataFrame()

# add a first column to the dataframe for beta
profits_df[!, "beta"] = zeros(length(betas))

# add columns to the dataframe for the profits per scenario
for i in 1:W
    profits_df[!, Symbol("profit_scenario_" * string(i))] = zeros(length(betas))
end


In [8]:
loop = 0

for beta in betas
    print("Beta: ", beta, "\n")
    loop += 1
    # Create a new model with GLPK solver
    model = Model(GLPK.Optimizer)
    unregister(model, :p_DA)
    #reset the model
    

    # Define the decision variables for hour
    @variable(model, p_DA[1:hours])
    @variable(model, delta[1:W,1:hours])
    @variable(model, delta_up[1:W,1:hours])
    @variable(model, delta_down[1:W,1:hours])
    @variable(model, zeta)
    @variable(model, eta[1:W])


    # Define the objective function
    @objective(model, Max, (1-beta) * sum(1/W *(
    scenarios[i][hour,"price"] * p_DA[hour]
    + delta_up[i,hour] * scenarios[i][hour,"price"] * (scenarios[i][hour,"grid_excess"]*0.9 + (1-scenarios[i][hour,"grid_excess"])*1)
    - delta_down[i,hour] * scenarios[i][hour,"price"] * (scenarios[i][hour,"grid_excess"]*1 + (1-scenarios[i][hour,"grid_excess"])*1.2)
    ) for i in 1:W, hour in 1:hours)
    + beta  * (zeta - (1/(1-alpha)) * sum(1/W .* eta[i] for i in 1:W)))

    # Define the constraints
    @constraint(model, [hour in 1:hours], p_DA[hour] <= 200)
    @constraint(model, [hour in 1:hours], p_DA[hour] >= 0)
    @constraint(model, [i in 1:W, hour in 1:hours], delta[i,hour] == scenarios[i][hour,"wind power"] - p_DA[hour])
    @constraint(model, [i in 1:W, hour in 1:hours], delta[i,hour] == delta_up[i,hour] - delta_down[i,hour])
    @constraint(model, [i in 1:W, hour in 1:hours], delta_down[i,hour] >= 0)
    @constraint(model, [i in 1:W, hour in 1:hours], delta_up[i,hour] >= 0)
    @constraint(model, [i in 1:W], eta[i] >= 0)
    @constraint(model, [i in 1:W], -1 * sum(1/W *(scenarios[i][hour,"price"] * p_DA[hour]
        + delta_up[i,hour] * scenarios[i][hour,"price"] * (scenarios[i][hour,"grid_excess"]*0.9 + (1-scenarios[i][hour,"grid_excess"])*1)
        - delta_down[i,hour] * scenarios[i][hour,"price"] * (scenarios[i][hour,"grid_excess"]*1 + (1-scenarios[i][hour,"grid_excess"])*1.2)) for hour in 1:hours) 
        + zeta - eta[i] <= 0)


    # Solve the optimization problem
    optimize!(model)

    exp_profit = value.(sum(1/W *(
        scenarios[i][hour,"price"] * p_DA[hour]
        + delta_up[i,hour] * scenarios[i][hour,"price"] * (scenarios[i][hour,"grid_excess"]*0.9 + (1-scenarios[i][hour,"grid_excess"])*1)
        - delta_down[i,hour] * scenarios[i][hour,"price"] * (scenarios[i][hour,"grid_excess"]*1 + (1-scenarios[i][hour,"grid_excess"])*1.2)
        ) for i in 1:W, hour in 1:hours))

    CVaR = (value.(zeta) - (1/(1-alpha)) * sum(1/W .* value.(eta[i]) for i in 1:W))

    # Print the termination status
    status = termination_status(model)
    if status == MOI.OPTIMAL
        #println("Optimal solution found")

        println("p_DA: ", value.(p_DA))
        
        # RETURN OBJECTIVE value
        println("Objective value: ", objective_value(model))
        println("Expected profit: ", exp_profit)
        println("CVaR: ", CVaR)
        #println("zeta: ", value.(zeta))
        #print sum of eta
        #println("sum of eta: ", sum(value.(eta)))
        #push!(objectiv_values, objective_value(model))
        #push!(exp_profits, exp_profit)
        #push!(CVaR_values, CVaR)
    else
        println("No optimal solution found")
    end

     #save the profits of each scenario for this beta in profits_df
    #  profits_df[loop, "beta"] = beta
    #  for i in 1:W
    #      profits_df[loop, Symbol("profit_scenario_" * string(i))] = value.(sum(1/W *(
    #         scenarios[i][hour,"price"] * p_DA[hour]
    #         + delta_up[i,hour] * scenarios[i][hour,"price"] * (scenarios[i][hour,"grid_excess"]*0.9 + (1-scenarios[i][hour,"grid_excess"])*1)
    #         - delta_down[i,hour] * scenarios[i][hour,"price"] * (scenarios[i][hour,"grid_excess"]*1 + (1-scenarios[i][hour,"grid_excess"])*1.2)
    #         ) for hour in 1:hours))
    #  end
end

Beta: 0.0
p_DA: [64.39741935483872, 14.389693451612905, 56.90564516129032, 50.954193548387096, 43.07419354838709, 45.95790322580645, 145.9269354516129, 13.34106441935484, 1.1038967741935486, 146.94499998387096, 21.6095805, 0.7419306451612904, 142.91516125806453, 50.27709674193549, 135.9887096612903, 53.12935479032258, 146.0248386935484, 149.60145158064518, 16.439596645161288, 152.74596772580645, 4.493516129032258, 49.94887096774193, 6.808774177419355, 149.57870966129033]
Objective value: 130697.43588726835
Expected profit: 130697.4358872683
CVaR: 6.415223594478769
Beta: 0.1
p_DA: [64.39741935483872, 14.389693451612905, 56.90564516129032, 50.954193548387096, 43.07419354838709, 45.95790322580645, 145.9269354516129, 13.34106441935484, 1.1038967741935486, 146.94499998387096, 21.6095805, 0.7419306451612904, 142.91516125806453, 50.27709674193549, 135.9887096612903, 53.12935479032258, 146.0248386935484, 149.60145158064518, 16.439596645161288, 152.74596772580645, 4.493516129032258, 49.94887096

In [ ]:
profit_df = DataFrame(scenario = Int[], profit = Float64[])

for i in 1:W
    profit = 0
    for hour in 1:hours
        profit += (scenarios[i][hour,"price"] * value(p_DA[hour])
        + scenarios[i][hour,"price"] * value(delta[i, hour])*((0.9.*scenarios[i][hour,"grid_excess"]) + 1.2.*(1-scenarios[i][hour,"grid_excess"])))
    end
    push!(profit_df, [i, profit])
end

In [17]:
#make a dataframe with the results
results = DataFrame(betas = betas, objective_values = objectiv_values, exp_profits = exp_profits, CVaR_values = CVaR_values)

#save the results
CSV.write("1_3_results/two_price_results.csv", results)

"1_3_results/two_price_results.csv"

In [8]:
# save the profits per scenario for each beta

CSV.write("1_3_results/two_price_profits.csv", profits_df)


"1_3_results/two_price_profits.csv"